# Submission with models ensemble

In [ ]:
model_name = time.strftime("%Y%m%d_%H%M%S") + '_models_ensemble'
model_path = os.path.join(WORKSPACE_DIR, 'models', model_name)
os.makedirs(model_path)
model_path

In [ ]:
ENSAMBLE_MODELS = [
    '20230522_154806',
    '20230522_171329',
    '20230522_172610',
    '20230522_173632',
    '20230522_200731',
    '20230522_193437',
    '20230522_203044',
]

In [ ]:
scores_ensemble = []
for m in ENSAMBLE_MODELS:
    scores_file_path = os.path.join(WORKSPACE_DIR, 'models', m, "scored_eval_samples.json")
    with open(scores_file_path) as file:
        scores_ensemble.append(json.load(file))

In [ ]:
assert all(len(s) == 10744 for s in scores_ensemble)
assert all(all(k in next(iter(s.values())).keys() for k in ['categories', 'osd']) for s in scores_ensemble)

In [ ]:
scored_eval_samples = dict()
for id in scores_ensemble[0].keys():
    scores_from_all_models = [s[id]['categories'] for s in scores_ensemble]
    categories = np.mean(scores_from_all_models, axis=0)
    categories_std=np.std(scores_from_all_models, axis=0)
    osd_mean = np.mean([s[id]['osd'] for s in scores_ensemble])
    osd = min(1, osd_mean + 5 * np.mean(categories_std))
    scored_eval_samples[id] = dict(
        categories=categories,
        categories_std=categories_std,
        osd=osd
    )

In [ ]:
max_category_scores = []
for id, sample_with_preds in scored_eval_samples.items():
    max_category_scores.append(max(sample_with_preds['categories']))

plt.hist(max_category_scores, bins=30)
plt.xlim([-0.05, 1.05])
plt.title(f'Highest category probability \n over all evaluation samples for model {model_name}')
plt.savefig(os.path.join(model_path, f'{model_name}_max_category_scores.png'))
plt.show()

In [ ]:
mean_categories_std = []
for id, sample_with_preds in scored_eval_samples.items():
    mean_categories_std.append(np.mean(sample_with_preds['categories_std']))

plt.hist(mean_categories_std, bins=30, cumulative=True)
plt.xlim([-0.05, 1.05])
plt.title(f'Mean category standard deviation \n over all evaluation samples for model {model_name}')
plt.savefig(os.path.join(model_path, f'{model_name}_mean_categories_std.png'))
plt.show()

In [ ]:
osd_scores = []
for id, sample_with_preds in scored_eval_samples.items():
    osd_scores.append(sample_with_preds['osd'])

plt.hist(osd_scores, bins=30)
plt.xlim([-0.05, 1.05])
plt.title(f'OSD probability \n over all evaluation samples for model {model_name}')
plt.savefig(os.path.join(model_path, f'{model_name}_osd_scores.png'))
plt.show()

## Get ranked predictions

In [ ]:
all_scores_per_category = defaultdict(list)
# all_scores_per_supercat = defaultdict(list)
categories_threshold = 0.4

for id, scores_per_output in tqdm(scored_eval_samples.items()):
    # categories
    ranked_classes = np.argsort(scores_per_output['categories'])[::-1]
    ranked_predicted_categories = []
    for class_index in ranked_classes:
        if scores_per_output['categories'][class_index] < categories_threshold and len(ranked_predicted_categories) > 0:
            break
        class_one_hot_array = np.zeros(len(ranked_classes))
        class_one_hot_array[class_index] = 1.0
        predicted_category = categories_encoder.inverse_transform(np.array([class_one_hot_array]))
        predicted_category = predicted_category[0][0]
        all_scores_per_category[predicted_category].append(scores_per_output["categories"][class_index])
        ranked_predicted_categories.append(predicted_category)
    assert len(ranked_predicted_categories) > 0
    scored_eval_samples[id]['ranked_predicted_categories'] = ranked_predicted_categories

    # # supercategories
    # ranked_supercats = np.argsort(scores_per_output['supercats'])[::-1]
    # ranked_predicted_supercats = []
    # for supercat_index in ranked_supercats:
    #     supercat_one_hot_array = np.zeros(len(ranked_supercats))
    #     supercat_one_hot_array[supercat_index] = 1.0
    #     predicted_supercat = supercats_encoder.inverse_transform(np.array([supercat_one_hot_array]))
    #     predicted_supercat = predicted_supercat[0][0]
    #     all_scores_per_supercat[predicted_supercat].append(scores_per_output['supercats'][supercat_index])
    #     ranked_predicted_supercats.append(predicted_supercat)
    # scored_eval_samples[id]['ranked_predicted_supercats'] = ranked_predicted_supercats

In [ ]:
num_predicted_categories = [len(s['ranked_predicted_categories']) for s in scored_eval_samples.values()]
plt.hist(num_predicted_categories, bins=30, cumulative=True)
plt.title(f'Number of predicted categories \n over all evaluation samples for model {model_name}')
plt.savefig(os.path.join(model_path, f'{model_name}_num_predicted_categories.png'))
plt.show()

## Examine the score distributions

### Categories

## Save results in submission file

In [ ]:
submission_ids = []
submission_categories = []
submission_osd_scores = []
for id, sample_with_preds in tqdm(scored_eval_samples.items()):
    submission_ids.append(id)
    submission_categories.append(sorted(sample_with_preds['ranked_predicted_categories']))
    submission_osd_scores.append(sample_with_preds['osd'])

In [ ]:
submission = pd.DataFrame({'id': submission_ids, 'categories': submission_categories, 'osd': submission_osd_scores})
submission.head()

In [ ]:
submission.to_csv(os.path.join(model_path, 'submission.csv'), index=False)

In [ ]:
save_current_ipynb(model_path)